## 7.2 – Sets
### Hashing
We introduced complexity and big O notation with searching lists. As we mentioned on Engage, we are now going to wrap back around and show how we can perform membership tests in (roughly) $O(1)$ time, by creating an implementation of the set data type. You already know what a set is from using Python sets like `{1, 2}`, but how is membership testing so efficient?

The idea hinges on the concept of a *hash function*. We alluded to this when we talked about overriding the `__eq__()` method, that it's good practice to override `__hash__()` at the same time. Now let's find out what that actually is.

A *hash function* (or just a *hash*) is used for mapping data of *variable* size into data of *fixed* size. Though, this definition is complicated, because every application that requires a hash function will have slightly different requirements. It will be easier to understand if we go through some examples.

For our set implementation, we want to accept any data type, but we need to refer to each item by some ID – some way of quickly checking whether two items are equal purely numerically. Hash values do not *necessarily* have to be *unique* in all implementations, but it makes things much easier if they are. If the hash function always returns a unique value for any input, it is called a *universal hash function*.

Writing good hash functions is hard. Python provides a function called `hash` which is extremely useful, though it only works on immutable types.

In [1]:
hash(1000)

1000

In [2]:
hash(0.1)

230584300921369408

In [3]:
hash("mystring")

8326158350310127125

In [4]:
hash(("mystring", 0.1))

-485402296581523010

In [5]:
hash(["cannnot be mutable"])

TypeError: unhashable type: 'list'

The `hash(…)` function calls the `.__hash__()` method of the object. If we are writing our own classes, the easiest way to define the hash method is to put the constituent parts (i.e. the attributes) into a tuple and hash this.

The Python model instructs us to ensure that if `.__eq__(…)` returns `True` between two objects, then they should have the same `.__hash__()` value. It does not say anything about the reverse implication, but again, truly unique hashes are hard to construct. 

This distinction will become less important soon anyway, so for now let's assume that the `hash` is always unique, and more usefully, always an integer.

### Hash Sets
Now let's use this hash function. Hopefully you've guessed how we're going to get $O(1)$ membership testing: we're going to use an array!

(Actually, the mechanism here is really similar to the dictionary, you might start spotting the parallels – we'll explain fully in the next notebook.)

Suppose you create an array which is reasonably large. Let's say it has `n` items, where `n` is much bigger than the number of items you are likely to store. You also have a way of turning objects into unique integers, the `hash(…)` function in Python. 

Now, use this function to find a location inside your array. There is no guarantee the integers are within range (`0` to `n-1`), but that's okay, we can use our old friend modulo. So the location of an object `x` is found with `hash(x) % n`.

This *is* a function which maps from variable sized data to fixed sized data! This is what some sources use as their definition of a “hash function” – so, confusingly, our hash function *uses* the `hash` function. Also note that this type of hash function *must* result in duplicate values. If `n = 1000`, there are only 1000 possible values for `hash(x) % 1000`, no matter how many possible values there are for `hash(x)`. We know that there are more than 1000 unique objects (there are more than 1000 integers!) so by the [pigeonhole principle](https://en.wikipedia.org/wiki/Pigeonhole_principle), some must be duplicates. But let's forget that for a second and assume `n` is so big that duplicates are really unlikely. 

So: given an object `x` we calculate `x_index = hash(x) % n`. Now we use position `x_index` of our array to indicate whether the set contains the object, and magically we get $O(1)$ cost for adding and membership testing, at the expense of a big area of memory (the array).

What do we store in the array? Various options – `True` and `False` might seem like an obvious one – but, when `x` is present I am going to store `x` itself, and otherwise I'll store `None`. I'll explain this choice later. Take a look:

In [6]:
class HashSet:
    def __init__(self, arr_size=100):
        self.arr = [None] * arr_size
        self.n = arr_size
        
    def add(self, x):
        x_index = hash(x) % self.n
        self.arr[x_index] = x
        
    def is_member(self, x):
        x_index = hash(x) % self.n
        if self.arr[x_index] is None:
            return False
        return True
    
    
my_set = HashSet()
my_set.add(15)
my_set.add(2348092348)
my_set.add("banana")


print(my_set.is_member(15))
print(my_set.is_member(2348092348))
print(my_set.is_member("banana"))

print(my_set.is_member("apple"))

True
True
True
False


This set implementation seems to work nicely! 

But you can probably spot a problem. What happens if there's a conflict?

In [7]:
print(my_set.is_member(9162857348))

True


We didn't add this element to the set, but we get a positive result. This is a problem.

We need some kind of *collision resolution*. Without getting too stuck in the weeds, a simple solution is to pick the *next* array location whenever we hit a collision. That means we need to be able to test whether the hash took us to the right object, hence why we are storing `x` in the array. Take a look at the updated code below.

In [8]:
class HashSet:
    def __init__(self, arr_size=100):
        self.arr = [None] * arr_size
        self.n = arr_size
        
    def add(self, x):
        x_index = hash(x) % self.n
        
        while self.arr[x_index] is not None and self.arr[x_index] != x:
            x_index = (x_index + 1) % self.n
            
        self.arr[x_index] = x
        
    def is_member(self, x):
        x_index = hash(x) % self.n
        
        while self.arr[x_index] is not None and self.arr[x_index] != x:
            x_index = (x_index + 1) % self.n
            
        if self.arr[x_index] is None:
            return False
        
        return True
    
    
my_set = HashSet()
my_set.add(15)
my_set.add(2348092348)
my_set.add("banana")

print(my_set.is_member(2348092348))
print(my_set.is_member(9162857348))

True
False


This technique is called *linear probing*. The new code is:
```python
while self.arr[x_index] is not None and self.arr[x_index] != x:
    x_index = (x_index + 1) % self.n
```

This conditional detects collisions: if the array value at position `x_index` is `None` or `x`, it will skip the loop entirely, otherwise it must be a collision, in which case it will keep going until one of those two things is true, which will tell us for sure whether the item is in the set. Since we use the same method for adding items into the array an testing for membership, we should be good.

This code could hit an infinite loop if the array is completely full! We could detect if we've looped round, but in practice don't actually need to, more on that in a second.

Instead, let's get back to analysis, do we still actually get $O(1)$ complexity? Certainly not in the worst case: hopefully it's clear we get linear behaviour when there are lots of collisions, because the probing has to keep scanning elements of the array. The proportion of occupied space in the array is called the *load factor*, and the higher this is, the higher the likelihood of a collision. Searches that result in a negative result must keep going until they find a blank space, so if the array is close to full, this can be very slow. 

Linear probing is especially negatively impacted if there are lots of items that happen to map into the same array position – a good hash function needs to be evenly spread across the array size, and since in Python `hash(x) = x` when `x` is an integer, it's possible you could get certain patterns of numbers much more often depending on what you are using the set for. This would be a good reason to write our own hash function from scratch. Or we can try to avoid this “grouping” with a different probing technique – for example we could take another application of a hash function to try to move into another area of the array.

Provided we keep the number of collisions low, the performance is very good, it is still basically $O(1)$ on average. Most hash set implementations will actually *resize* once they reach a certain load factor, and this is especially important with linear probing because it has exponentially worse performance after it hits a load factor of roughly 75-80%. Resizing is costly, of course. We must allocate the new array, then move every item out of the old one and into the new, which likely means recomputing the hash function on every item.

### Hashing Revisited
Since we can, let's be really barbaric and make a class which totally breaks the hash function. You'll see in the code below that Python's inbuilt set functionality does not get tripped up, so it must be doing the same thing we are here: storing the true value of each item, and comparing them with `==` (`__eq__(…)`) before informing us whether the set contains the value or not.

In [9]:
class BadNumber:
    def __init__(self, number):
        self.number = number
        
    def __hash__(self):
        return 1
    
    def __eq__(self, other):
        if type(self) == type(other):
            return self.number == other.number
        else:
            return NotImplemented
        
    def __str__(self):
        return "" + self.number
        
        
        
my_set = {BadNumber(4), BadNumber(15), BadNumber(-300000)}
print(hash(BadNumber(4)))
print(hash(BadNumber(0)))
print(BadNumber(4) in my_set)
print(BadNumber(0) in my_set)

1
1
True
False


And we shouldn't really be surprised. It's a necessity to use an array of some fixed length, and so no matter how you manipulate the result of `hash(…)`, collisions will happen. This is why we can be a bit cagey about whether `hash(…)` does or needs to return unique values – ideally yes, in practice for hash sets, it does not matter.

Of course hashes are used in other applications too. For example, when you sign up for an account on a website, your password is hashed before being stored in the database. Then, when you sign in, whatever you type in the password field is put through the same hash and compared to the stored version. In this case the hash is designed to make the value *more* complicated, not less. The idea is that the hash is so complicated you could not reverse it to work out the original password, if you stole it from the database or during transit. Clearly, for this hash, uniqueness is paramount, otherwise someone could log in with a different password!

(This is also a good reason why you should never trust a website that will offer to recover/email you your password – they should not be storing it in plain text or any other recoverable format to begin with!)

## What Next?
Interestingly, in most languages and textbooks, the hash set is really just an afterthought from the *hash table*, which is another name for the *dictionary*. Everything we just talked about: collisions, linear probing – these are usually discussed in the context of that data structure. It's only a small modification, so head back to Enage to move onto that in the next section.